<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Build and Save a Spark model to WSL Repository</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
</table>

This notebook walks you through these steps:
 
- Access the data
- Cleanse data for analysis
- Explore data
- Build a classification model
- Save the model in the DSX repository with associated meta data


### Step 1: Download Data as Spark Dataframe from Local Storage

Read the three Mortgage files from local storage - Insert them as SPARK Dataframes

In [ ]:
import os
from pyspark.sql import SQLContext
# Add asset from file system
df_data_1 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/MortgageCustomer.csv', header='true', inferSchema = 'true')

In [ ]:
# Add asset from file system
df_data_2 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Property.csv', header='true', inferSchema = 'true')

In [ ]:
# Add asset from file system
df_data_3 = SQLContext(sc).read.csv(os.environ['DSX_PROJECT_DIR']+'/datasets/Default.csv', header='true', inferSchema = 'true')

In [ ]:
customer = df_data_1
property = df_data_2
default = df_data_3

In [ ]:
customer.cache()
property.cache()
default.cache()

View the data within the three data frames created above. 

In [ ]:
customer.show(5)
#property.show(5)
#default.show(5)

### Step 2: Merge Files

In [ ]:
# Join the Customer, Property and Default tables together with ID being the key field.
# Keep all fields from the Customer and Default tables but only SalePrice and Location from the Property table

merged = customer.join(property, customer['ID'] == property['ID'])\
                   .join(default, customer['ID']==default['ID'])\
                   .select(customer['ID'],customer['Income'],customer['AppliedOnline'],customer['Residence'],\
                           customer['Yrs at Current Address'],customer['Yrs with Current Employer'],customer['Number of Cards'],\
                           customer['Creditcard Debt'],customer['Loans'],customer['Loan Amount'],\
                           property['SalePrice'], property['Location'], default['MortgageDefault'])

# Preview  5 rows
merged.toPandas().head()

### Step 3: Simple Data Preparation - Rename some columns and ensure correct data types 
This step is to remove spaces from columns names

In [ ]:
merged = merged.withColumnRenamed("Yrs at Current Address", "YearCurrentAddress").withColumnRenamed("Yrs with Current Employer","YearsCurrentEmployer")\
                .withColumnRenamed("Number of Cards","NumberOfCards").withColumnRenamed("Creditcard Debt","CCDebt").withColumnRenamed("Loan Amount", "LoanAmount")
merged.toPandas().head(3)

Check data types and re-cast numeric fields to **Integers**

In [ ]:
merged.dtypes

In [ ]:
merged = merged.select(merged.ID.cast("integer"),merged.Income.cast('integer'),merged.AppliedOnline,merged.Residence,\
                   merged.YearCurrentAddress.cast('integer'),merged.YearsCurrentEmployer.cast('integer'),\
                   merged.NumberOfCards.cast('integer'),merged.CCDebt.cast('integer'),merged.Loans.cast('integer'),\
                   merged.LoanAmount.cast('integer'),merged.SalePrice,merged.Location,merged.MortgageDefault)
merged.dtypes

### Step 4: Data Exploration

1) Obtain some data shape summaries in terms of number of fields and records <br>
2) Perform some exploratory analysis of distributions, scatterplots using two different graphics packages

#### Feel free to play around with the charts to explore other features that are in the data set

In [ ]:
print "There are " + str(len(merged.toPandas())) + " records and " + str(len(merged.toPandas().columns)) + " fields in the dataset."

In [ ]:
import brunel
df = merged.toPandas()
%brunel data('df') bar x(Residence) y(Income) mean(CCDebt) color(MortgageDefault) stack tooltip(Income) | x(YearCurrentAddress) y(YearsCurrentEmployer) point color(MortgageDefault) tooltip(YearCurrentAddress, YearsCurrentEmployer) :: width=1100, height=400 

In [ ]:
from pixiedust.display import *
display(merged)

### Step 5: Build the Spark pipeline and the Random Forest model
"Pipeline" is an API in SparkML that's used for building models.
Additional information on SparkML: https://spark.apache.org/docs/2.0.2/ml-guide.html

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# Prepare string variables so that they can be used by the algorithm
stringIndexer1 = StringIndexer(inputCol='AppliedOnline', outputCol='AppliedOnlineEncoded')
stringIndexer2 = StringIndexer(inputCol='Residence',outputCol='ResidenceEncoded')
stringIndexer3 = StringIndexer(inputCol='MortgageDefault', outputCol='label')

# Instanciate the algorithm
rf=RandomForestClassifier(labelCol="label", featuresCol="features")


# Pipelines API requires that input variables are passed in  a vector
assembler = VectorAssembler(inputCols=["Income", "AppliedOnlineEncoded", "ResidenceEncoded", "YearCurrentAddress", "YearsCurrentEmployer", "NumberOfCards", \
                                       "CCDebt", "Loans", "LoanAmount", "SalePrice", "Location"], outputCol="features")

pipeline = Pipeline(stages=[stringIndexer1, stringIndexer2, stringIndexer3, assembler, rf])

In [ ]:
# Split data into train and test datasets
train, test = merged.randomSplit([80.0,20.0], seed=6)

In [ ]:
# Build model based upon the pipeline defined in the above cell
model = pipeline.fit(train)

### Step 6: Score the test data set

In [ ]:
results = model.transform(test)
results.toPandas().head(3)

### Step 7: Model Evaluation 

In [ ]:
print 'Precision model1 = {:.2f}.'.format(results.filter(results.label == results.prediction).count() / float(results.count()))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label", metricName="areaUnderROC")
print 'Area under ROC curve = {:.2f}.'.format(evaluator.evaluate(results))

In [ ]:
evaluatorRF = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluatorRF.evaluate(results)

print("Accuracy = %g" % accuracy)

### Step 8: Save Model in ML repository


In [ ]:
from dsx_ml.ml import save

model_name = "Predict Mortgage Default LOS"
save(name = model_name,
     model = model,
     algorithm_type = 'Classification',
     test_data = test)

## Summary:
You are now at the end of Lab 1 and should have successfully:
- Loaded data 
- Performed basic data preparation on the loaded data
- Explored the data graphically
- Build a Spark Model in the form of a pipeline
- Evaluated the model for accuracy levels
- Stored the model in to the IBM ML environment in readiness for deployment

---------------------------------------------------------------------------------
Author Information:<br>
**Stephen Groves** based on an original notebook created by Sidney Phoon, IBM USA
<br/>
steve.groves@uk.ibm.com
<br/>
9th April 2018